In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from cebra import CEBRA
from PIL import Image
import cv2
import torch
import pickle

In [ ]:
from logging import exception
def process_brain(path):
  try:
    brain_seq = cv2.imreadmulti(path)[1]
    brain_seq = np.array(brain_seq)
    brain_mask = (np.sum(brain_seq, axis=0) > 0)
    flat_seq = (brain_seq[:, brain_mask])
    return flat_seq.astype(float)
  except:
    print(np.shape(brain_seq))

In [ ]:
# iterate through all brain data and process it
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
import os
neural_data = []
name_data = []

filepath = "2020_11_9_MV1_run_brain"
for iter, file in enumerate(os.listdir(filepath)):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         neural_data.append(process_brain(filepath + '/' + filename))
         name_data.append(filename)
         continue
     else:
         continue

In [ ]:
image_data = []
filepath = "2020_11_9_MV1_run_behavior"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         image_data.append(cv2.imreadmulti(filepath + '/' + filename)[1])
         continue
     else:
         continue

In [ ]:
multi_cebra_model = CEBRA.load('cebra_multi_model2.pt')

In [ ]:
def flatten_data(data):
    data_flat = np.squeeze(data[0])
    for x in data[1::]:
        data_flat = np.concatenate((data_flat, np.squeeze(x)))
    return data_flat

In [ ]:
import cebra
neural_embedding = []
for run, data in enumerate(neural_data):
  try:
    neural_embedding.append(multi_cebra_model.transform(data, session_id=run))
  except:
    print(run)

In [ ]:
embedding_train = neural_embedding[:(int)(282 * 0.8)]
embedding_pred = neural_embedding[225::]

In [ ]:
def flatten_data(data):
    data_flat = np.squeeze(data[0])
    for x in data[1::]:
        data_flat = np.concatenate((data_flat, np.squeeze(x)))
    return data_flat

In [ ]:
image_data_flat = flatten_data(image_data[:(int)(282 * 0.8)])
neural_embeddings_train_flat = flatten_data(embedding_train)
neual_embeddings_pred_flat = flatten_data(embedding_pred)

In [ ]:
import sklearn.metrics
image_decoder = cebra.KNNDecoder(n_neighbors=5, metric="cosine")
image_decoder.fit(neural_embeddings_train_flat, (image_data_flat.astype(int)))
image_pred = image_decoder.predict(neual_embeddings_pred_flat)

In [ ]:
shape_list = [np.shape(x)[0] for x in embedding_pred]
gen_video_list = []
index = 0
for shape in shape_list:
    gen_video_list.append(image_pred[index : index + shape])
    index += shape